### Seleção de modelos


In [15]:
def metricas(scores,target):
    return pd.DataFrame({
        'KS'   : [ks2(scores,target)],
        'ROC'  : [roc_auc_score(y_score=scores,y_true=target)],
        'GINI' : [2 * roc_auc_score(y_score=scores,y_true=target) - 1]
          })
    
def ks2(score, alvo):
    b2 = pd.crosstab(score, alvo, rownames = ['score']).reset_index()
    if len(np.unique(alvo)) > 1:
        freq_bom = b2.iloc[:,1]/sum(b2.iloc[:,1])
        freq_mau = b2.iloc[:,2]/sum(b2.iloc[:,2])
        ac_bom = np.cumsum(freq_bom)
        ac_mau = np.cumsum(freq_mau)
        diff = np.abs(ac_bom - ac_mau).round(7)
        ks = diff.max() * 100
    else:
        ks = 0
    return ks

def tabela_geral(modelos, X,y,nomes):
  lista = []
  noms = []
  for i in range(len(modelos)):
    tab = metricas(modelos[i].predict_proba(X)[:,0],y)
    lista.append(tab)
    nom = nomes[i]
    noms.append(nom)   
  tabela_g = pd.concat([lista[0],lista[1]],ignore_index=True)
  tabela_g['modelo']= noms
  return tabela_g[['modelo', 'KS','ROC','GINI']]  

def plot_scores(df, score_base='Score_solucao'):
    num_bins = 10 
    plt.figure(figsize=(8, 4))
    plt.hist(df[score_base], num_bins, facecolor='blue', alpha=0.4,label=str(score_base))
    plt.legend(prop={'size': 10})
    plt.xlabel("Scores")
    plt.ylabel("Frequencia")
    tittle = str('Histogram')
    plt.title("{}".format(str(tittle)))
    plt.show()

import matplotlib.pyplot as plt
def plot_graph(x,y,x_nome,y_nome,tittle):
  fig = plt.figure(figsize=(15, 4))
  ax = fig.add_axes([0,0,1,1])
  ax.bar(x,y,label=str(x_nome))
  plt.legend(prop={'size': 10})
  plt.xlabel(str(x_nome))
  plt.ylabel(str(y_nome))
  plt.title(str(tittle))
  plt.show()

In [22]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder #, SimpleImputer
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')
     

# Carregando o dataset
# URL do conjunto de dados Titanic
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'

# Importando o conjunto de dados
df = pd.read_csv(url)
df = df.rename(columns={'Survived': 'Target'})
df = df.drop(columns=['PassengerId', 'Name', 'Ticket'])

# Selecionando as variáveis
features = [x for x in list(df.columns) if x not in ['Target']] # Exclui a coluna target
target = ['Target']

# Pré-processamento
# Imputação de valores missing
colunas_numericas = list(df.select_dtypes(include=['number']).columns)
colunas_numericas.remove('Target')
imputer = SimpleImputer(strategy='median')

# imputer.fit(df[features])

# Transformação de variáveis categóricas
categorical_features = list(df.select_dtypes(include=['category', 'object']).columns)
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
# onehot_encoder.fit(df[categorical_features])

# Pré-processamento em pipeline
preprocessor = ColumnTransformer([
    ('imputer', imputer, colunas_numericas),
    ('onehot', onehot_encoder, categorical_features)
])
df_processado = pd.DataFrame(preprocessor.fit_transform(df[features]))


In [10]:
df_processado

,0,1,2,3,4,5,6,7,8,9,...,149,150,151,152,153,154,155,156,157,158
0,3.0,22.0,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1.0,38.0,1.0,0.0,71.2833,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,3.0,26.0,0.0,0.0,7.9250,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,1.0,35.0,1.0,0.0,53.1000,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,3.0,35.0,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2.0,27.0,0.0,0.0,13.0000,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
887,1.0,19.0,0.0,0.0,30.0000,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
888,3.0,28.0,1.0,2.0,23.4500,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
889,1.0,26.0,0.0,0.0,30.0000,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [23]:
X = df_processado
y = df['Target']
resultados = []

numero_repeticoes = 5
algoritmos = []

for k in range(0,numero_repeticoes):
        metricas_cada_validacao = []
        i = 0
        skf = StratifiedKFold(n_splits=2+k, random_state=None)
        for train_index, val_index in skf.split(X,y):
            i += 1
            X_train, X_test = X.iloc[train_index], X.iloc[val_index]
            y_train, y_test = y.iloc[train_index], y.iloc[val_index]
            modelo_01 = XGBClassifier(subsample= 0.6, silent= False, reg_lambda= 1.0, n_estimators= 100,
                          min_child_weight= 7.0, max_depth= 20, learning_rate= 0.01, gamma= 0, 
                          colsample_bytree= 0.5, colsample_bylevel= 0.9)
            modelo_01.fit(X_train,y_train)
            algoritmos.append(modelo_01)
            metricas_cada_validacao.append(metricas(modelo_01.predict_proba(X)[:,1],y))
            print("Resultado da Validação nº",i,":",metricas(modelo_01.predict_proba(X)[:,0],y))
        resultados.append(pd.DataFrame({'Validacao '+str(k):pd.concat(metricas_cada_validacao).apply(np.mean,axis=0)}))
pd.concat(resultados,axis=1)
     

Resultado da Validação nº 1 :          KS       ROC      GINI
0  56.07324  0.150028 -0.699944
Resultado da Validação nº 2 :          KS      ROC     GINI
0  59.32317  0.13613 -0.72774
Resultado da Validação nº 1 :          KS      ROC     GINI
0  56.54779  0.13854 -0.72292
Resultado da Validação nº 2 :          KS       ROC      GINI
0  58.54185  0.129236 -0.741529
Resultado da Validação nº 3 :          KS       ROC      GINI
0  58.66168  0.135147 -0.729705
Resultado da Validação nº 1 :          KS       ROC      GINI
0  58.14879  0.132785 -0.734429
Resultado da Validação nº 2 :          KS       ROC      GINI
0  59.68747  0.127323 -0.745353
Resultado da Validação nº 3 :          KS       ROC      GINI
0  58.89176  0.130431 -0.739138
Resultado da Validação nº 4 :          KS       ROC      GINI
0  58.58019  0.131691 -0.736618
Resultado da Validação nº 1 :          KS       ROC      GINI
0  58.39804  0.132514 -0.734973
Resultado da Validação nº 2 :          KS       ROC      GINI
0  61.

,Validacao 0,Validacao 1,Validacao 2,Validacao 3,Validacao 4
KS,57.698205,57.917107,58.827052,59.675964,60.386508
ROC,0.856921,0.865692,0.869442,0.870426,0.872709
GINI,0.713842,0.731385,0.738885,0.740852,0.745418
